In [8]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import numpy as np
import pandas as pd
import seaborn as sns
import shared_utils
from scipy.stats import zscore

# set_option to increase max rows displayed to 200, to see entire df in 1 go/
pd.set_option("display.max_rows", 200)

# function to display df info
def df_peek(df):
    display(type(df), df.shape, df.dtypes)

Read in Raw Data
17C and 17B via excels

In [7]:
# Params to read in dgs data
url_17c = 'gs://calitp-analytics-data/data-analyses/bus_procurement_cost/17c compiled-Proterra Compiled Contract Usage Report .xlsx'
url_17b = 'gs://calitp-analytics-data/data-analyses/bus_procurement_cost/17b compiled.xlsx'

sheet_17c = 'Proterra '
sheet_17b = 'Usage Report Template'

def read_excel(url, sheet):
    df = pd.read_excel(url, sheet_name=sheet)
    
    return df

dgs_17c = read_excel(url_17c, sheet_17c)
dgs_17b = read_excel(url_17b, sheet_17b)

In [ ]:
merge data frames

In [15]:
display(dgs_17c.shape, dgs_17b.shape)

(74, 25)

(33, 25)

In [13]:
merge = pd.merge(dgs_17b, dgs_17c, how='outer',on='Purchase Order Number')

In [14]:
# check work
# number of rows is correct
# will need to work on merge columns

df_peek(merge)

pandas.core.frame.DataFrame

(107, 49)

Supplier Contract Usage ID_x                                                       float64
Ordering Agency Name_x                                                              object
State (S) or Local (L) agency_x                                                     object
Purchasing Authority Number                    (for State departments)_x           float64
Agency Billing Code_x                                                              float64
Purchase Order Number                                                               object
Purchase Order Date_x                                                       datetime64[ns]
Delivery Date_x                                                                     object
Contract Line Item Number (CLIN)                (RFP ID)_x                         float64
UNSPSC Code\n(Version 10)_x                                                        float64
Manufacturer Part Number (OEM #)_x                                                  object

Data Cleaning and QC

Export Cleaned data
save out as parquet

Initial Summary Stats